In [1]:
import numpy as np
import weakref

In [2]:
class Variable:
    """ステップ１のVariableクラスを変更"""
    __array_priority__ = 200
    def __init__(self, data, name=None):
        if data is not None: # dataがNoneでなく
            if not isinstance(data, np.ndarray): # ndarrayインスタンスでもない場合
                raise TypeError("{} is not supported".formatf(type(data)))
        self.data = data
        self.name = name # 変数に名前を付ける
        self.grad = None
        self.creator = None # 変数視点では関数は生みの親（creator）
        self.generation = 0
    
    def set_creator(self, func):
        """creatorを設定"""
        self.creator = func 
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # 逆伝播の最初の勾配1を生成
            
        funcs = []
        
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
            
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() # 1.関数を取得
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
                if not retain_grad:
                    for y in f.outputs:
                        y().grad=None
            
            """x, y = f.input, f.output # 2.関数の入出力を取得
            x.grad = f.backward(y.grad)
            
            if x.creator is  not None:
                funcs.append(x.creator) # 4.自分より一つ前の変数のbackwardメソッドをリストに追加"""
            
    def cleargrad(self):
            self.grad = None
        
    @property # メソッドをインスタンス変数として扱える
    def shape(self):
        """多次元配列の形状を取得"""
        return self.data.shape
    
    @property 
    def ndim(self):
        """次元数を取得"""
        return self.data.ndim
    
    @property 
    def size(self):
        """要素数を取得"""
        return self.data.size
    
    @property 
    def dtype(self):
        """データ型を取得"""
        return self.data.dtype
    
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace("\n", "\n" + " " * 9)
        return "variable(" + p + ")"
    
    """def __mul__(self, other):
        return mul(self, other)"""

In [3]:
class Function:
    """
    ステップ２のFunctionクラスを変更
    新たに
    ・微分を行う逆伝播の機能
    ・forwardメソッドを呼ぶ際に，入力されたVariableインスタンスを保持する機能
    """
    def __call__(self, *inputs):# f=Function()としたとき，f(...)と書くことでメソッドを呼び出せる
        inputs = [as_variable(x) for x in inputs]
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # ndarrayとして返す
        
        # 逆伝播に用いる情報を記憶
        if Config.enable_backprop: # 逆伝播モードがオンの時だけ逆伝播に用いるコードを実行
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self) # 出力変数に生みの親を覚えさせる
            self.inputs = inputs # 入力された変数を覚える
            self.outputs = [weakref.ref(output) for output in outputs] # 出力も覚える
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        raise NotImplementedError() # このメソッドは継承して実装すべきことをアピール
    
    def backward(self, gy):
        raise NotImplementedError() 

In [4]:
class Square(Function):
    """backwardメソッドを追加"""
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    

class Exp(Function):
    """backwardメソッドを追加"""
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
    
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [5]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

# ステップ18 逆伝播の有効モードと無効モードの切り替え

In [6]:
class Config:
    enable_backprop = True

In [7]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    # yieldの前で前処理
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
     # yieldの後ろで後処理
    finally:
        setattr(Config, name, old_value)

In [8]:
def no_grad():
    return using_config("enable_backprop", False)

# ステップ20　演算子のオーバーロード

In [9]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy
    
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0
    
def mul(x0, x1):
    return Mul()(x0, x1)

In [10]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)
"""+, * の計算で関数を呼び出せる"""
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [11]:
with no_grad():
    # 勾配を使わないときはno_grad()の中で実行する
    x = Variable(np.array(2.0))
    y = square(x)

In [12]:
x = Variable(np.array([3.0]))
y = np.array([3.0]) + x
y

variable([6.])

# ステップ24 複雑な関数の実装

In [14]:
from dezero import Variable

In [15]:
def sphere(x, y):
    z = x ** 2 + y ** 2
    return z


def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z


def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

In [16]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0
